In [1]:
import os
import sys
import math
import operator
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Conv2D
from multiprocessing import Pool
from keras.utils import Sequence
from keras.layers import Flatten
from keras.layers import MaxPool2D
from keras.models import Sequential
from collections import OrderedDict
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import load_model, save_model
os.chdir("D:\\fyp\Testing")

Using TensorFlow backend.


In [ ]:
class ImageDataGenerator(Sequence):
    
    def __init__(self, x_metadata, y_metadata, batch_size, crop_size):
        self.x = x_metadata
        self.y = y_metadata
        self.batch_size = batch_size
        self.cp = crop_size
    
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([np.transpose(np.array(tif.imread(file_name), dtype=int)/255.0,
                                     (1,2,0))[self.cp:-self.cp,self.cp:-self.cp,:]
                        for file_name in batch_x]), np.array(batch_y)      

class CNN_Model:
    
    def __init__(self, directory):
        
        self.path = directory
      #  df = pd.read_csv("occurrences_train.csv",low_memory=False)
        with open("D:\\fyp\Dataset\\save.pkl","rb") as f:
            hd = pkl.load(f)
        with open("D:\\fyp\Dataset\\save_test.pkl","rb") as f:
            hdt = pkl.load(f)
        self.train_metadata_y=[]
        self.train_pathdata_x = []
        self.train_seq_y = []
        self.test_pathdata_x = []
        self.test_seq_y = []
        for extra in hd.keys():
            for cls in hd[extra].keys():
                for order in hd[extra][cls].keys():
                    for family in hd[extra][cls][order].keys():
                        for genus in hd[extra][cls][order][family].keys():
                            for specie in hd[extra][cls][order][family][genus]:
                                path=self.path+"\\"+cls+"\\"+order+"\\"+family+"\\"+genus+"\\"+specie
                              #  print(path)
                                for im in os.listdir(path):
                                    self.train_pathdata_x.append(path+"\\"+im)
  
        
        
        np.random.shuffle(self.train_pathdata_x)
   
        self.classes=list(range(0, 768))
        for p in self.train_pathdata_x:
            y = p.split("\\")
            c = y[3]
            o = y[4]
            f = y[5]
            g = y[6]
            s = y[7]
            self.train_seq_y.append(np.array([[c],[o],[f],[g],[s]]))
        

            
    
            
    def model_create(self):
        
        classifier = Sequential()
        # Step 1 - Convolution
        classifier.add(TimeDistributed(Conv2D(filters=96, kernel_size=(2, 2), activation = 'relu'), input_shape=(5,32,32,33)))
        classifier.add(TimeDistributed(Conv2D(filters=256, kernel_size=(2, 2), activation = 'relu')))
        classifier.add(TimeDistributed(Conv2D(filters=256, kernel_size=(3, 3), activation = 'relu')))
        classifier.add(TimeDistributed(Conv2D(filters=512, kernel_size=(3, 3), activation = 'relu')))
        #classifier.add(Conv2D(filters=, kernel_size=(2, 2), activation = 'relu'))
        classifier.add(TimeDistributed(MaxPool2D(pool_size = (2, 2))))
        # Step 3 - Flattening
        classifier.add(TimeDistributed(Flatten()))
        # Step 4 - LSTM
        print(classifier.output_shape)
      #  classifier.add(Reshape((1,432640)))
        classifier.add(LSTM(100, return_sequences=True))
        # Step 5 - Full connection
        classifier.add(Dense(768, activation = 'relu'))
        # Compiling the CNN
        classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy')
        classifier.summary()
        return classifier
    
    def fit_generator(self, num_epochs=10, batch_size=32, crop_size=16):        
    
        classifier = self.model_create()
        train_data = ImageDataGenerator(self.train_pathdata_x,self.train_pathdata_x, 64, 16)
        history = classifier.fit_generator(train_data, epochs=num_epochs, use_multiprocessing=False,shuffle=True)
        classifier.save("D:\\fyp\\Dataset\\code\\Models\\RNN_5.h5")

        scores = classifier.evaluate_generator(train_data, use_multiprocessing=False)

In [ ]:
ob = CNN_Model("D:\\fyp\\Testing")

In [ ]:
ob.fit_generator()

In [ ]:
model=load_model("D:\\fyp\CNN_data\Selected\Code\Models\RNN_Model5.h5")
batch_x=["D:\\fyp\CNN_data\Selected\patchTrain\Anacamptis pyramidalis\\patch_30.tif"]
def preprocess(): 
    return np.array([np.transpose(np.array(tif.imread(file_name), dtype=int),
                                     (1,2,0))[16:-16,16:-16,:]
                        for file_name in batch_x])     
predictions=preprocess()
probabilities= model.predict(predictions)
print(probabilities[0]))

In [2]:
model=load_model("D:\\fyp\CNN_data\\Selected\Code\Models\RNN_model1.h5")
def preprocess(imgpath): 
    return np.array([np.transpose(np.array(tif.imread(file_name), dtype=int),
                                     (1,2,0))[16:-16,16:-16,:]
                        for file_name in imgpath])    
path="D:\\fyp\CNN_data\\patchTrain"
summation =0 ;
a=[]
correct_prob=0
counter=0;

for folder in os.listdir(path):

    folder_path=path+"\\"+folder;
    for next_path in os.listdir(folder_path):
        img_path= folder_path+"\\"+next_path
        batch_x=[img_path];
        predictions=preprocess(batch_x)
        probabilities= model.predict(predictions)
        a=probabilities[0].tolist()
        correct_prob=a[counter]
        a.sort()
        rank=a.index(correct_prob)
        rank+=1;
        summation+=1/rank
    counter+=1
print("Mean reciprocal rank=", summation/3524)  #Total number of images

0
Mean reciprocal rank= 0.0031987575309562165


In [4]:
model=load_model("D:\\fyp\CNN_data\\Selected\Code\Models\RNN_model5.h5")
def preprocess(imgpath): 
    return np.array([np.transpose(np.array(tif.imread(file_name), dtype=int),
                                     (1,2,0))[16:-16,16:-16,:]
                        for file_name in imgpath])    
path="D:\\fyp\CNN_data\Selected\\patchTrain"
summation =0 ;
a=[]
correct_prob=0
counter=0;

for folder in os.listdir(path):

    folder_path=path+"\\"+folder;
    for next_path in os.listdir(folder_path):
        img_path= folder_path+"\\"+next_path
        batch_x=[img_path];
        predictions=preprocess(batch_x)
        probabilities= model.predict(predictions)
        a=probabilities[0].tolist()
        correct_prob=a[counter]
        a.sort()
        rank=a.index(correct_prob)
        rank+=1;
        summation+=1/rank
    counter+=1
print("Mean reciprocal rank=", summation/566)  #Total number of images

Mean reciprocal rank= 0.22580767289247805
